# 7. Hypertuning and Aggregating
Ce notebook a pour but d'améliorer la prédiction de la localisation des devices.  


* La première étape consiste à réaliser une <u>*correction des bases outliers*</u>.  
* La deuxième étape consiste à <u>*retirer des devices difficiles à prédire*</u> des données d'entrainement. 
* La troisième étape repose sur une <u>*hyperparameter optimization*</u> des meilleurs modèles retenus.
* La dernière étape réalise une <u>*méthode d'aggregating*</u> des modèles tunés 


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Import des librairies de Nicolas 
from IpyTools import *
from IotTools import *
pd.options.mode.chained_assignment = None  # default='warn'


# Import des modèles à utiliser
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import ExtraTreeRegressor

from sklearn.model_selection import GridSearchCV

In [ ]:
df_mess_train = pd.read_csv('mess_train_list.csv')
df_mess_test = pd.read_csv('mess_test_list.csv')
pos_train = pd.read_csv('pos_train_list.csv')
listOfBs = np.union1d(df_mess_train.bsid.unique(),df_mess_test.bsid.unique())

In [ ]:
X_train = df_mess_train

## Fonction de protection réécrivant toutes les bases en dehors d'un certain cadre 

In [ ]:
X_train=Correct_Bases(df_mess_train)

Nous avons 27 bases outliers


## Suppression des devices difficiles à prédire

In [ ]:
X_train = X_train[~X_train.did.isin([476598,476896,476256,476513,476889,476248,473288,476327,476836])]
X_train.shape

(35668, 8)

## Création de la matrice des features

In [ ]:
df_feat, id_list=feat_mat_const(X_train, listOfBs)

y_full = ground_truth_const(X_train, pos_train, id_list)
y_full.shape,df_feat.shape

((4876, 3), (4876, 273))

## Tuning des hyperparamètres

Afin d'améliorer les modèles retenus, une optimisation des hyperparamètres en utilisant un GridSearchCV.

In [ ]:
def evaluation_model(model_lat,model_long, cv, df_feat,y_full) :
    '''Return the score used to evaluate the model'''
    y_pred_long = cross_val_predict(model_long, df_feat, y_full.lng, cv=cv, n_jobs=-1)
    y_pred_lat = cross_val_predict(model_lat, df_feat, y_full.lat, cv=cv,n_jobs=-1)
    err_vec = Eval_geoloc(y_full.lat , y_full.lng, y_pred_lat, y_pred_long)
    score = np.percentile(err_vec, 80)
    return score

In [ ]:
def gridsearch_multiple_models(df_feat,cv,param,model):
    '''Function that returns the best model for the longtitude and latitude'''
    model_long = GridSearchCV(model, param,cv=cv,n_jobs=-1)
    model_long.fit(df_feat, y_full.lng)
    
    model_lat = GridSearchCV(model, param,cv=cv,n_jobs=-1)
    model_lat.fit(df_feat, y_full.lat)
    
    return model_lat, model_long

In [ ]:
data = {'Model': [],  'Score_pre_tuning' : [], 'Score_post_tuning' : [], 'Hyperparams_lat' : [],  'Hyperparams_long' : [] } 

def dataframe_summary(name_model,pre_score,post_score,params_long,params_lat,data):
    '''Function that returns a Pandas DataFrame and the dictionnary used to make this DF
    This DataFrame gives information about the score of a model and its hyperparameters'''
    data["Model"].append(name_model)
    data["Score_pre_tuning"].append(pre_score)
    data["Score_post_tuning"].append(post_score)
    data['Hyperparams_lat'].append(params_lat)
    data['Hyperparams_long'].append(params_long) 
    return data

In [ ]:
models_description = [
    {"Model_name" : "XGBoost Regressor", "Model" : xgb.XGBRegressor() ,"params" : {'max_depth':[4,5,6],'n_estimators' : [50,100,200],'learning_rate' : [0.1,0.2],'booster' : ['gbtree'],'gamma' : [0,0.001,0.01],'subsample' : [0.8,0.9,1], 'criterion' : ('mse','mae'), 'max_features' : ('auto', 'sqrt', 'log2')}},
    {"Model_name" : "ExtraTree Regressor", "Model" : ExtraTreeRegressor(), "params" :  {'max_depth':[4,5,6,10,12,15,20],'n_estimators' : [50,100,150,200]} },
    {"Model_name" : "GradientBoosting Regressor", "Model" : GradientBoostingRegressor() , "params" : {'max_depth':[4,5,6],'n_estimators' : [50,100,200],'learning_rate' : [0.1,0.2],'subsample' : [0.8,0.9,1]}},
    {"Model_name" :"RandomForest Regressor" , "Model" :  RandomForestRegressor(), "params" : {'max_depth':[4,5,6,8,10,15],'n_estimators' : [50,100,200]}},
    {"Model_name" : "Bagging Regressor", "Model" : BaggingRegressor(), "params" : {'n_estimators' : [10,50,100] }}
]

In [ ]:
%%time
cv = 3
for model_desc in models_description :
    print("debut",model_desc["Model_name"])
    pre_score = evaluation_model(model_desc["Model"],model_desc["Model"], 3, df_feat,y_full)
    model_lat, model_long = gridsearch_multiple_models(df_feat,cv,model_desc["params"],model_desc["Model"])
    post_score = evaluation_model(model_lat.best_estimator_,model_long.best_estimator_, 10, df_feat,y_full)
    data = dataframe_summary(model_desc["Model_name"],pre_score,post_score,model_long.best_params_,model_lat.best_params_,data)
data

In [ ]:
df_results = pd.DataFrame(data).tail(5)
df_results

,Model,Score_pre_tuning,Score_post_tuning,Hyperparams_lat,Hyperparams_long
5,XGBoost Regressor,3.164077,2.634562,"{'booster': 'gbtree', 'gamma': 0, 'learning_ra...","{'booster': 'gbtree', 'gamma': 0.001, 'learnin..."
6,ExtraTree Regressor,2.875616,2.566386,"{'max_depth': 15, 'n_estimators': 150}","{'max_depth': 15, 'n_estimators': 50}"
7,GradientBoosting Regressor,2.917346,2.708313,"{'learning_rate': 0.1, 'max_depth': 4, 'n_esti...","{'learning_rate': 0.2, 'max_depth': 4, 'n_esti..."
8,RandomForest Regressor,2.892368,2.665724,"{'max_depth': 8, 'n_estimators': 50}","{'max_depth': 15, 'n_estimators': 50}"
9,Bagging Regressor,2.959867,2.614221,{'n_estimators': 100},{'n_estimators': 100}


**Conclusion :**
* On observe une amélioration significative de la prédiction en hypertunant nos modèles
* On observe également que les hyperparamètres ne sont pas forcément les mêmes pour les deux outputs (longitude, latitude)

### Sauvegarde des résultats  
Le résultats seront stockés dans un fichier csv pour être utilisé dans d'autres notebooks.

In [ ]:
df_params = df_results[["Model","Hyperparams_lat","Hyperparams_long"]]
df_params["lng"] = df_params["Hyperparams_long"]
df_params["lat"] = df_params["Hyperparams_lat"]
df_params = df_params.drop(columns=["Hyperparams_lat","Hyperparams_long"])
df_params.to_csv("best_params.csv",index_label = False)

In [ ]:
## Apercu du fichier sauvegardé
df_params = pd.read_csv("best_params.csv")

### Récupération des résultats  
Une fonction a été ajouté dans notre fichier *IotTools.py* qui retourne les hyperparamètres selon le modèle et le type de coordonnées.  
Exemple : 

In [ ]:
get_hyperparameter("RandomForestRegressor","lng")

{'criterion': 'mae',
 'max_depth': 10,
 'max_features': 'auto',
 'n_estimators': 25}

## Aggregation des modèles

Afin d'améliorer notre prédiction, on utilisera [VotingRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingRegressor.html) prenant en compte les résultats de l'ensemble des modèles (en moyennant l'ensemble des prédictions)

In [ ]:
from sklearn.ensemble import VotingRegressor

In [ ]:
def get_tuned_estimators(coord) : 
    return [('RandomForestRegressor_lat', RandomForestRegressor(**get_hyperparameter("RandomForestRegressor",coord))),
      ('ExtrasTreeRegressor', ExtraTreeRegressor(**get_hyperparameter("ExtraTreeRegressor",coord))),
     ('GradientBoostingRegressor', GradientBoostingRegressor(**get_hyperparameter("GradientBoostingRegressor",coord))),
     ('XGBRegressor', xgb.XGBRegressor(**get_hyperparameter("XGBRegressor",coord))),
     ('BaggingRegressor', BaggingRegressor(**get_hyperparameter("BaggingRegressor",coord)))]

In [ ]:
estimators_lat = get_tuned_estimators("lat")
estimators_lng = get_tuned_estimators("lng")

In [ ]:
reg_lat = VotingRegressor(estimators=estimators_lat,n_jobs=-1)
reg_long = VotingRegressor(estimators=estimators_lng, n_jobs=-1)
evaluation_model(reg_lat,reg_long, 10, df_feat,y_full)

2.553526

On remarque que cette technique de vote donne un meilleur résultat que les modèles précédents.  

Cependant, <u>l'entrainement est assez chronophage</u>    
Solution possible : **Sérialiser** le modèle une fois entrainé (en format *.pkl*, par exemple) pour une utilisation ultérieure.